In [1]:
import pandas as pd
import numpy as np
from src import util
from src.process2004_2017 import get_processed_enade_2014_2017
from src import config
from src import subject_manipulation

In [2]:
subject_df = subject_manipulation.get_processed_subject_df()

In [3]:
subject_df = subject_df.loc[subject_df['ano'] == 2017]
subject_df.head()

,idquestao,ano,curso,prova,tipoquestao,conteudo1,conteudo2,conteudo3
0,1,2017.0,CC,Geral,Discursiva,NaN,NaN,NaN
1,2,2017.0,CC,Geral,Discursiva,NaN,NaN,NaN
2,3,2017.0,CC,Geral,Objetiva,NaN,NaN,NaN
3,4,2017.0,CC,Geral,Objetiva,NaN,NaN,NaN
4,5,2017.0,CC,Geral,Objetiva,NaN,NaN,NaN


In [4]:
subjects = util.get_subjects(subject_df)
subjects

array(['Algoritmos e estruturas de dados',
       'Arquitetura de computadores e sistemas operacionais',
       'Banco de dados', 'Compiladores',
       'Computação gráfica e processamento de imagem',
       'Engenharia de software e interação homem-computador',
       'Fundamentos e técnicas de programação',
       'Inteligência artificial e computacional',
       'Lógica e matemática discreta',
       'Paradigmas de linguagens de programação', 'Redes de computadores',
       'Sistemas digitais', 'Sistemas distribuídos',
       'Teoria da computação', 'Teoria dos grafos',
       'Ética, computador e sociedade'], dtype=object)

In [5]:
enade_2017_df = get_processed_enade_2014_2017("data/enade/enade2017/3.DADOS/MICRODADOS_ENADE_2017.txt")
enade_2017_df.head()

,QUESTAO_1_NOTA,QUESTAO_2_NOTA,QUESTAO_3_NOTA,QUESTAO_4_NOTA,QUESTAO_5_NOTA,QUESTAO_6_NOTA,QUESTAO_7_NOTA,QUESTAO_8_NOTA,QUESTAO_9_NOTA,QUESTAO_10_NOTA,...,QUESTAO_32_NOTA,QUESTAO_33_NOTA,QUESTAO_34_NOTA,QUESTAO_35_NOTA,QUESTAO_36_NOTA,QUESTAO_37_NOTA,QUESTAO_38_NOTA,QUESTAO_39_NOTA,QUESTAO_40_NOTA,TP_PRES
55274,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,222.0
55275,72,95,100,100,0,100,100,100,0,100,...,0,100,100,0,100,100,0,0,0,555.0
55276,67,71,100,100,0,100,100,100,100,0,...,100,100,0,0,100,100,0,0,0,555.0
55277,67,BRANCO,100,100,100,100,100,100,0,100,...,100,0,0,0,100,0,0,0,0,555.0
55278,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,222.0


In [6]:
new_mapping = enade_2017_df["TP_PRES"].map({222:"Ausente", 555:"Presente"}).values
enade_2017_df.loc[:, "TP_PRES_LABEL"] = new_mapping
enade_2017_df["TP_PRES_LABEL"].value_counts()

Presente    34
Ausente     14
Name: TP_PRES_LABEL, dtype: int64

In [7]:
enade_2017_df = enade_2017_df.loc[enade_2017_df["TP_PRES_LABEL"] == "Presente"]

In [8]:
enade_2017_df = util.add_all_score_subjects(enade_2017_df, subject_df, objective=True)

(34,)
(34,)
(34,)
(34,)
(34,)
(34,)
(34,)
(34,)
(34,)
(34,)
(34,)
(34,)
(34,)
(34,)
(34,)
(34,)
(34,)
(34,)
(34,)
(34,)
(34,)
(34,)
(34,)
(34,)
(34,)
(34,)


/home/renan/git/KDD-HigherEduBR/src/util.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  question_score[zero_score_index] = 0


(34,)
(34,)
(34,)
(34,)
(34,)
(34,)
(34,)
(34,)
(34,)
(34,)
(34,)
(34,)
(34,)
(34,)
(34,)
(34,)
(34,)
(34,)
(34,)
(34,)
(34,)
(34,)
(34,)
(34,)
(34,)
(34,)
(34,)
(34,)


In [9]:
# medias dos alunos por cada tema, junto com o número de questões
num_questions = []
mean_acertos_by_subject = []
std_acertos_by_subject = []
column_zero_subject = []
column_geq_one_subject = []
column_all_subject = []
num_alunos = enade_2017_df.shape[0]

for subject in subjects:
    subject_questions = util.get_subject_valid_questions(subject, 
                                                                 subject_df,
                                                                 enade_2017_df,
                                                                 just_objective=True)
    num_obj_subject_questions = len(subject_questions)
    num_questions.append(num_obj_subject_questions)
    
    mean_acertos_subject = enade_2017_df[f"ACERTOS_OBJ_{subject}"].mean()
    mean_acertos_by_subject.append(mean_acertos_subject)
    
    std_acertos_subject = enade_2017_df[f"ACERTOS_OBJ_{subject}"].std()
    std_acertos_by_subject.append(std_acertos_subject)
    
    zero_subject = enade_2017_df[f"ACERTOS_OBJ_{subject}"].value_counts()[0] / num_alunos
    column_zero_subject.append(zero_subject)
    
    geq_one_subject = (enade_2017_df[f"ACERTOS_OBJ_{subject}"] >= 1).value_counts()[True] / num_alunos
    column_geq_one_subject.append(geq_one_subject)
    
    all_subject = (enade_2017_df[f"ACERTOS_OBJ_{subject}"] == num_obj_subject_questions).value_counts()
    if True in all_subject.index:
        column_all_subject.append(all_subject[True] / num_alunos)
    else:
        column_all_subject.append(0)
    
    
subjects_labels = [f"SCORE_OBJ_{x}" for x in subjects]

mean_by_subject = enade_2017_df[subjects_labels].mean().values

data = np.array([mean_by_subject, num_questions]).T

display_df = pd.DataFrame(data=data, index=subjects, 
                          columns=["Nota %", "Nº Questões"])
display_df["Nº Questões"] = display_df["Nº Questões"].astype(int)
display_df["Média Acertos"] = mean_acertos_by_subject
display_df["Desvio Padrão Acertos"] = std_acertos_by_subject
display_df["% de Zeros"] = np.array(column_zero_subject)*100
display_df["% de Alunos que acertaram pelo menos uma questão"] = np.array(column_geq_one_subject)*100
display_df["% de Alunos que acertaram todas"] = np.array(column_all_subject)*100

display_df = display_df.sort_values(by=["Nota %"]).round(2)
display_df


,Nota %,Nº Questões,Média Acertos,Desvio Padrão Acertos,% de Zeros,% de Alunos que acertaram pelo menos uma questão,% de Alunos que acertaram todas
Teoria da computação,13.24,2,0.26,0.45,73.53,26.47,0.00
Banco de dados,30.88,2,0.62,0.55,41.18,58.82,2.94
Compiladores,32.35,1,0.32,0.47,67.65,32.35,32.35
Algoritmos e estruturas de dados,35.29,4,1.41,1.08,23.53,76.47,2.94
Arquitetura de computadores e sistemas operacionais,35.29,3,1.06,0.81,23.53,76.47,5.88
Computação gráfica e processamento de imagem,41.18,1,0.41,0.50,58.82,41.18,41.18
Sistemas distribuídos,41.18,2,0.82,0.80,41.18,58.82,23.53
Teoria dos grafos,44.12,1,0.44,0.50,55.88,44.12,44.12
Fundamentos e técnicas de programação,47.06,1,0.47,0.51,52.94,47.06,47.06
Lógica e matemática discreta,47.06,1,0.47,0.51,52.94,47.06,47.06


In [24]:
objective_questions = subject_manipulation.get_objective_questions(subject_df)
enade_2017_df_objective = enade_2017_df[[f"QUESTAO_{i}_NOTA" for i in objective_questions]]
num_blank = 0
num_deletion = 0
for column in enade_2017_df_objective.columns:
    if "BRANCO" in enade_2017_df_objective[column].values:
        num_blank += enade_2017_df_objective[column].value_counts()["BRANCO"]
    if "RASURA" in enade_2017_df_objective[column].values:
        num_deletion += enade_2017_df_objective[column].value_counts()["RASURA"]
percentage_blank = num_blank * 100 / np.prod(enade_2017_df_objective.shape)
percentage_blank = round(percentage_blank, 2)



print(f"{num_blank} questão marcada em branco, o que equivale a "
      f"{percentage_blank}% das questões objetivas")
print(f"{num_deletion} questões rasuradas")

1 questão marcada em branco, o que equivale a 0.08% das questões objetivas
0 questões rasuradas
